In [1]:
# SCRIPT to MAKE TARGETS FROM NL_THEMES in metadata

In [2]:
# Import pandas
import pandas as pd
 
# reading csv file
df=pd.read_csv("../data/CLA_meta_from_2018.csv") 

### THIS script does EVERYTHING (for NL)
 Extract keywords from CLA data and put them in a dataframe with the text added

In [3]:
# FORGET THE ABOVE,
# THIS does EVERYTHING (for NL)
# Extract keywords from CLA data and put them in a dataframe with the text added

#make new dataframe combining CLA_meta with extracted keywords
from os import listdir
import pandas as pd
import re

# reading csv file
df=pd.read_csv("../data/CLA_meta_from_2018.csv") 

# Add custom stopwords
from nltk.corpus import stopwords
stopwords = stopwords.words('dutch')

custom_stopwords=['waarvoor','wegens','sommige','betreffende','maatregel','stelsel','excl','aanv','adv','artikel','hoofdstuk','2020','2019','2018','2021','2022','uren']
stopwords.extend(custom_stopwords)
print (stopwords)

from rake_nltk import Metric, Rake
rake_nltk_var = Rake(language='dutch',stopwords=stopwords,include_repeated_phrases=False,ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO)
import re


# Go through all the files

input_path='../processed_data/NL'
file_list=listdir(input_path)

doc_list=[]
for file in file_list:
    if 'NL_' in file:
        document_id=file[3:-4]
    else:
        document_id=file[:-4]
    doc_list.append(document_id) 

# Make a dataframe which will have document_id and the keywords in seperate columns
df_target_full=pd.DataFrame()

for document_id in doc_list:

    # get the themes_text (NL)
    themes_text=df.loc[df['filename'].str.contains(document_id),'themes_nl'].values.astype(str).tolist()

    # Check if value is filled and extract keywords
    if pd.isnull(df.loc[df['filename'].str.contains(document_id),'themes_nl'].values)!=True:
        themes_text = re.sub("\)\\\\","",themes_text[0])
        themes_text = re.sub("\)\.","",themes_text)
        themes_text = re.sub(" \)","",themes_text)
        themes_text = re.sub("\),",",",themes_text)
        themes_text = re.sub("-,",",",themes_text)
        themes_text = re.sub(",-",",",themes_text)
        rake_nltk_var.extract_keywords_from_text(themes_text)
        keyword_extracted = rake_nltk_var.get_ranked_phrases()

    theme_keywords=df.loc[df['filename'].str.contains(document_id),'themes_nl'].values[0]

    # Keyword_extracted is a list, so split them into seperate words for the columns 
    doc_keywords=[document_id] 
    targets=set(keyword_extracted)

    # First column will be document_id, so put it as first element
    doc_keywords=[document_id]

    # Exclude errors like 'e' as keyword
    for x in targets:
        if len(x)>3:
            doc_keywords.append(x)

    # Put the keywords to a dataframe and transpose to fit with full_target dataframe
    df_target=pd.DataFrame(doc_keywords)
    df_target=df_target.transpose()

    df_target_full=pd.concat([df_target_full,df_target],ignore_index=True)


['de', 'en', 'van', 'ik', 'te', 'dat', 'die', 'in', 'een', 'hij', 'het', 'niet', 'zijn', 'is', 'was', 'op', 'aan', 'met', 'als', 'voor', 'had', 'er', 'maar', 'om', 'hem', 'dan', 'zou', 'of', 'wat', 'mijn', 'men', 'dit', 'zo', 'door', 'over', 'ze', 'zich', 'bij', 'ook', 'tot', 'je', 'mij', 'uit', 'der', 'daar', 'haar', 'naar', 'heb', 'hoe', 'heeft', 'hebben', 'deze', 'u', 'want', 'nog', 'zal', 'me', 'zij', 'nu', 'ge', 'geen', 'omdat', 'iets', 'worden', 'toch', 'al', 'waren', 'veel', 'meer', 'doen', 'toen', 'moet', 'ben', 'zonder', 'kan', 'hun', 'dus', 'alles', 'onder', 'ja', 'eens', 'hier', 'wie', 'werd', 'altijd', 'doch', 'wordt', 'wezen', 'kunnen', 'ons', 'zelf', 'tegen', 'na', 'reeds', 'wil', 'kon', 'niets', 'uw', 'iemand', 'geweest', 'andere', 'waarvoor', 'wegens', 'sommige', 'betreffende', 'maatregel', 'stelsel', 'excl', 'aanv', 'adv', 'artikel', 'hoofdstuk', '2020', '2019', '2018', '2021', '2022', 'uren']


In [4]:
df_target_full.head(-3)

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,88
0,10202-2018-012766,bedrijfstoeslag,commercenachtarbeid,organisatie,loonkloof,alle premies,jaarbasis,cheque,syndicale vormingopleiding,ploegen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10202-2020-013175,bedrijfstoeslag,loonkloof,alle premies,jaarbasis,cheque,syndicale vormingopleiding,ploegen,pensioen,vergoedingen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10205-2018-004963,bedrijfstoeslag,commercenachtarbeid,eindeloopbaandagentijdskrediet,organisatie,herstructurering,alle premies,jaarbasis,ploegen,pensioen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10206-2019-003872,alle premies,groepsverzekeringen,aanvullende pensioenen,vergoedingen,actieve werknemer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10206-2020-000814,commercenachtarbeid,organisatie,alle premies,jaarbasis,ploegen,pensioen,groepsverzekeringen,aanvullende pensioenen,vergoedingen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5523,341-2021-005291,betaalde verlofdag,compensatiedag,ecocheques,vakantiegeld,alle premies,premie,cheque,groepsverzekeringen,aanvullende pensioenen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5524,341-2021-015425,onderneming,sector,alle premies,premie eigen,sport,vergoedingen,cultuurcheques,actieve werknemer,geschenken,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5525,341-2021-015811,bedrijfstoeslag,lonen,werkloosheid,swtindexeringsbepalingen,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5526,341-2021-015813,persoonlijke redenen,ouderschapsverlof,verlof,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Write it to a temporary dataframe
df_target_full.to_csv('../csv/CLA_deg_to_ratio_filtered_meta_keywords.csv', sep=";", index=False) 

In [6]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

def clean_text(text):
  text = text.lower()
  text = re.sub("[^a-zA-Z\'\-éòóôëè]", " ", text) 
  return " ".join(word_tokenize(text)[:])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Theranet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# Add the text of the files (CLA) to the dataframe
import os
def add_CLA_text_to_df():
    input_path=os.path.join('..','processed_data','NL')
    file_list=listdir(input_path)
    doc_list=[]

    for file in file_list:
        if 'NL_' in file:
            document_id=file[3:-4]
        else:
            document_id=file[:-4]
        doc_list.append(document_id) 

    df=pd.read_csv('../csv/CLA_deg_to_ratio_filtered_meta_keywords.csv',sep=';')

    for document_id in doc_list:

        try: 
            with open(os.path.join(input_path,f'NL_{document_id}.txt'),encoding="utf-8") as f:
                text = f.read() 
                df.loc[df['0']==document_id,'text']=clean_text(text)
        except FileNotFoundError:
            with open(os.path.join(input_path,f'{document_id}.txt'),encoding="utf-8") as f:
                text = f.read() 
                df.loc[df['0']==document_id,'text']=clean_text(text)


    df.reset_index(inplace=True)
    df.to_csv('../csv/NL_document_targets_deg_to_ratio_filtered.csv', sep=";", index=False)
    
add_CLA_text_to_df()
